<a href="https://colab.research.google.com/github/joohoshin/TensorflowAI/blob/main/11_2_HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fasion mnist 정확도를 향상 시켜봅시다

In [1]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [2]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']               

In [3]:
num_classes = 10

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

x_train = x_train/255
x_test = x_test/255

In [4]:
from tensorflow.keras import layers, models

# model = models.Sequential()
# model.add(layers.Conv2D(32,(5,5), padding='valid', activation='relu', input_shape=(28,28,1)))
# model.add(layers.MaxPool2D(2,2))
# model.add(layers.Flatten())
# model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(10, activation='softmax'))
# model.summary()

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# epochs = 20
# history = model.fit(x_train, y_train, batch_size=64,
#           epochs=epochs, validation_data=(x_test, y_test))

In [5]:
!pip install -q -U keras-tuner

In [6]:
import kerastuner as kt

In [7]:
from tensorflow.keras import layers, models

def model_builder(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(32,(5,5), padding='valid', activation='relu', input_shape=(28,28,1)))
    model.add(layers.MaxPool2D(2,2))
    model.add(layers.Flatten())
    
    # Dense Layer Output size 바꾸기 위해서 변수 선언
    hp_units = hp.Int('units', min_value = 512, max_value = 2048, step = 512)
    
    # units에 변수로 입력
    model.add(layers.Dense(units = hp_units, activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    model.summary()
    
    # Adam Optimizer 학습률 선택지
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    
    # Optimizer에 학습률 변수를 넣기 위해서 keras.optimizers.Adam 사용
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss='categorical_crossentropy', 
                  metrics = ['accuracy'])
    return model

In [8]:
# 인스턴스 만들기
tuner = kt.RandomSearch(model_builder,
                     objective = 'val_accuracy', 
                     max_trials = 10)    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2359808   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 2,365,770
Trainable params: 2,365,770
Non-trainable params: 0
______________________________________________

In [9]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 10 Complete [00h 00m 36s]
val_accuracy: 0.9180999994277954

Best val_accuracy So Far: 0.9180999994277954
Total elapsed time: 00h 07m 40s
INFO:tensorflow:Oracle triggered exit


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [11]:
best_hps.values

{'units': 512, 'learning_rate': 0.001}

In [12]:
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2359808   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 2,365,770
Trainable params: 2,365,770
Non-trainable params: 0
______________________________________________

https://www.tensorflow.org/tutorials/keras/keras_tuner